In [77]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import keras
from tensorflow.keras.layers import Activation, Convolution2D, Dropout, Conv2D
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
class myCallback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('val_accuracy')>0.90):

      print("\nReached 98% accuracy so cancelling training!")

      self.model.stop_training = True

In [69]:
dataset_path = "/home/arjun/DM/fer2013.csv"
image_size=(48,48)

In [70]:
def load_fer2013():
        data = pd.read_csv(dataset_path)
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'),image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).to_numpy()
        return faces, emotions

In [71]:
faces, emotions = load_fer2013()

In [72]:
num_samples, num_classes = emotions.shape

In [79]:
X=faces
y=emotions
patience = 50

In [80]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model

In [81]:
X = X/255.0
X=np.array(X)
y=np.array(y)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
model = mini_XCEPTION((48, 48, 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 46, 46, 8)    72          input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 46, 46, 8)    32          conv2d_18[0][0]                  
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 46, 46, 8)    0           batch_normalization_32[0][0]     
____________________________________________________________________________________________

In [82]:
callbacks = myCallback()
history=model.fit(X, y, batch_size=32, epochs=40, validation_split=0.2,callbacks=[callbacks,reduce_lr])

Epoch 1/40
898/898 [==============================] - 150s 165ms/step - loss: 1.8188 - accuracy: 0.3176 - val_loss: 1.5152 - val_accuracy: 0.4351
Epoch 2/40
898/898 [==============================] - 140s 156ms/step - loss: 1.4212 - accuracy: 0.4791 - val_loss: 1.3562 - val_accuracy: 0.4967
Epoch 3/40
898/898 [==============================] - 140s 156ms/step - loss: 1.2817 - accuracy: 0.5316 - val_loss: 1.3832 - val_accuracy: 0.5059
Epoch 4/40
898/898 [==============================] - 141s 157ms/step - loss: 1.1888 - accuracy: 0.5543 - val_loss: 1.2607 - val_accuracy: 0.5319
Epoch 5/40
898/898 [==============================] - 142s 158ms/step - loss: 1.1258 - accuracy: 0.5831 - val_loss: 1.3272 - val_accuracy: 0.5085
Epoch 6/40
840/898 [===========================>..] - ETA: 8s - loss: 1.0827 - accuracy: 0.6016

KeyboardInterrupt: 